In [1]:
import torchvision
from torchvision import models
from torchvision.transforms import transforms
from tqdm import tqdm
import torch
import ImageData
import numpy as np
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import time
import json

/home/vislab-001/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "densenet"
model_image_size = 224
# vit = models.vit_l_16(models.ViT_L_16_Weights.IMAGENET1K_V1)

In [3]:
class DenseNet(torch.nn.Module):
    def __init__(self, numClasses: int, softmax:bool = True):
        super(DenseNet, self).__init__()

        self.densenetBase = torchvision.models.densenet121(weights='DEFAULT')
        self.features = self.densenetBase.features
        self.relu = torch.nn.ReLU()
        self.avgPool = torch.nn.AdaptiveMaxPool2d((1,1))
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 216),
            torch.nn.ReLU(),
            torch.nn.Linear(216, 4)
        )
        
        for param in list(self.densenetBase.parameters())[:-1]:
            param.requires_grad = True
        # for param in self.inceptionBase.parameters():
        #     print(param.requires_grad)

        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        features = self.features(x)
        out = self.relu(features)
        out = self.avgPool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out

In [4]:
model = DenseNet(4).to(device)
# print(*list(model.children())[:-1])

In [5]:
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [6]:
batch_size = 16
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.RandomResizedCrop(size=(model_image_size, model_image_size), antialias=True), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainDataLoader, validDataLoader, testDataLoader, unseenDataLoader = ImageData.getImagesDataloaders("../ArtiFact/", transforms = transform, batchSize=batch_size)

In [9]:
dataset = trainDataLoader.dataset

In [5]:
from torch.utils.data import Subset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

/home/vislab-001/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
trainIdxs, _ = train_test_split(range(len(dataset.imagePaths)), train_size=110, random_state=42)

NameError: name 'dataset' is not defined

In [20]:
ValIdxs = trainIdxs[100:]
trainIdxs = trainIdxs[:100]

In [48]:
len(trainIdxs)

100

In [21]:
trainSub = Subset(dataset, trainIdxs)
valSub = Subset(dataset, ValIdxs)

In [3]:
trainSubDataloader = DataLoader(trainSub, batch_size=32, shuffle=True)
valSubDataloader = DataLoader(valSub, batch_size=32, shuffle=True)

NameError: name 'DataLoader' is not defined

In [2]:
def evaluate_on_data(model, dataloader, dirty: bool = False):
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        total_loss = 0
        
        num_correct = 0.0
        num_correct_dirty = 0.0

        num_samples = 0.0
        for data in tqdm(dataloader, desc="Eval: "):
            image, label = data
            label = label.to(device)
            image = image.to(device)
            outputs = model(image)
            
            dirtyLabel = torch.where(label > 1, torch.tensor(1, dtype = torch.int32).to(device), label)

            
            
            loss = criterion(outputs, label)
            total_loss += loss.item()
            argMax = torch.argmax(outputs, 1)

            # print("pred")
            # print(outputs)
            # print(argMax)
            # print("gt")
            # print(label)
            for i in range(len(label)):
                
                if label[i] == argMax[i]:
                    num_correct += 1

                if (dirtyLabel[i] == argMax[i]) or (dirtyLabel[i] == 1 and argMax[i] > 0):
                    num_correct_dirty += 1

                num_samples += 1
                    
                
                
    return total_loss / len(dataloader), num_correct / num_samples, num_correct_dirty / num_samples

In [8]:
num_epochs = 10
count = 0
valid_loss_array = np.zeros(num_epochs)
valid_acc_array = np.zeros(num_epochs)
valid_acc_dirty_array = np.zeros(num_epochs)

train_loss_array = np.zeros(num_epochs)
for epoch in range(num_epochs):
    batch_count = 0
    for data in tqdm(trainDataLoader, desc="Training: "):
        
        image, label = data
        
        label = label.to(device)
        image = image.to(device)

        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        count += 1
        # print(loss)
            
        
    valid_loss, valid_acc, valid_acc_dirty = evaluate_on_data(model, validDataLoader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Valid Loss: {valid_loss}, Valid ACC: {valid_acc}, Dirty Valid ACC: {valid_acc_dirty}')
    valid_loss_array[epoch] = valid_loss
    train_loss_array[epoch] = loss.item()
    valid_acc_array[epoch] = valid_acc
    valid_acc_dirty_array[epoch] = valid_acc_dirty


Training:   0%|          | 0/117213 [00:00<?, ?it/s]

Eval: 100%|██████████| 6170/6170 [37:11<00:00,  2.76it/s] 


Epoch [1/10], Training Loss: 0.3937, Valid Loss: 0.8057969989227901, Valid ACC: 0.6563598601894535, Dirty Valid ACC: 0.7070766425206423


Eval: 100%|██████████| 6170/6170 [18:12<00:00,  5.65it/s]


Epoch [2/10], Training Loss: 0.6292, Valid Loss: 0.6904794076297619, Valid ACC: 0.7244921736487513, Dirty Valid ACC: 0.7701838812623474


...

Eval: 100%|██████████| 6170/6170 [13:47<00:00,  7.46it/s]


Epoch [6/10], Training Loss: 0.2724, Valid Loss: 0.5412681179311326, Valid ACC: 0.7914087432247606, Dirty Valid ACC: 0.8262803302770882


Eval: 100%|██████████| 6170/6170 [13:47<00:00,  7.45it/s]


Epoch [7/10], Training Loss: 0.4825, Valid Loss: 0.5321472648319877, Valid ACC: 0.7957347652094625, Dirty Valid ACC: 0.8297452003444608


Training:   2%|▏         | 2633/117213 [08:09<5:50:37,  5.45it/s]

In [ ]:
test_loss, test_acc, test_acc_dirty = evaluate_on_data(model, testDataLoader)
unseen_loss, unseen_acc, unseen_acc_dirty = evaluate_on_data(model, unseenDataLoader)

testingData = {
    "test_loss" : test_loss,
    "test_acc" : test_acc,
    "test_acc_dirty": test_acc_dirty,
    "unseen_loss" : unseen_loss, 
    "unseen_acc" : unseen_acc, 
    "unseen_acc_dirty": unseen_acc_dirty
}

# Serializing json
json_object = json.dumps(testingData, indent=4)
 
# Writing to sample.json
with open(f"./results/{model_name}_testing.json", "w") as outfile:
    outfile.write(json_object)

In [1]:
with open(f"./results/{model_name}_valid_loss.npy", 'wb') as f:
    np.save(f, valid_loss_array)
    
with open(f"./results/{model_name}_valid_acc.npy", 'wb') as f:
    np.save(f, valid_acc_array)

with open(f"./results/{model_name}_valid_dirty_acc.npy", 'wb') as f:
    np.save(f, valid_acc_dirty_array)
    
with open(f"./results/{model_name}_train.npy", 'wb') as f:
    np.save(f, train_loss_array)

torch.save(model.state_dict(), f"./savedModels/{model_name}Params.pth")

NameError: name 'model_name' is not defined